# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [2]:
using JuMP, Gurobi
using DataFrames, CSV

include(joinpath("capex_model", "expansion_tools.jl"))

save (generic function with 1 method)

In [35]:
function basic_capex(buses, lines, gens, loads, variability, P)
    """
    Function to solve DC OPF problem 
    Inputs:
        gens -- dataframe with generator info and costs
        lines -- dataframe with transmission lines info
        buses -- dataframe with bus types and loads
    """   
    #### HELPER FUNCTIONS ####
    # Select a line column using i and j 
    line = (i, j, col) -> lines[lines_map[(i, j)], col]
    # Subset of all lines connected to bus i
    J = i -> lines.t_bus[lines.f_bus .== i]
    # Select all generators in G connected to bus i
    sel = (G, i) -> collect(G)[gens[collect(G),:bus] .== i]
    # Return the last time index with circular wrapping
    function prev(t)
        """
        Returns the t-1 index using wrapping to ensure
        that if t-1 ∉ P then the largest t ∈ P is returned.
        """
        P_i = findfirst(P_i -> t in P_i, P)
        shift = first(P[P_i])
        p = length(P[P_i])
        return mod1(t - shift, p) + shift - 1
    end
    
    ##### SETS ####
    G = Set(gens[:, :gen_id])            # G: Set of all generators
    G_s = Set(gens[gens.ess .== 1, :gen_id])      # G_s: Set of all energy storage system (ESS) generators
    G_c = Set(gens[gens.canidate .== 1, :gen_id]) # G_c: Set of generators that can be expanded
    G_sc = intersect(G_s, G_c)        # G_sc = G_c ∩ G_s
    
    N = 1:nrow(buses)            # N: Set of all network nodes
    L = collect(zip(             # L: Set of all lines/branches
        lines.f_bus, lines.t_bus))

    
    T = 1:ncol(loads)            # T: Set of all hours
                                 # P: Set of representative periods
    W = ones(ncol(loads)) ### TODO fix this!!

    #### GLOBALS ####
    # Mapping from (i, j) tuple to line index l
    lines_map = Dict(zip(L, 1:length(L)))
    
    #### PARAMETERS ####
    θlim = π*(60/180)  # Absolute max angle limit in rad
    slack_bus = 416    # Set slack bus to one with many CTs
    
    storage_hrs = 4    # Storage Hours * Power Capacity = Energy capacity ∀ g ∈ G_s
    η_charge = 0.97    # Charging efficiency  
    η_discharge = 0.91 # Discharging efficiecy

    # Toy parameters for a limited investment in solar at each bus
    shed_cost = 1000   # Cost of load sheding
    budget = 0
    solar_cost = 14.75      #0.1475
    battery_cost = 14.75
    
    ########################
    #####     MODEL    #####
    ########################
    
    CATS_Model = Model(() -> Gurobi.Optimizer())
    #set_optimizer_attribute(CATS_Model, "NumericFocus", 2)
    
    # Decision variables   
    @variables(CATS_Model, begin
        GEN[G,T] ≥ 0       # Generation of each generator 
        SHED[N,T] ≥ 0      # Load sheading at bus N
        THETA[N,T]         # Voltage phase angle of bus
        FLOW[L,T]          # Flows between all pairs of nodes
        SOC[G_s,T] ≥ 0     # ESS state of charge
        CHARGE[G_s,T] ≥ 0  # Chargeing of ESS 
        CAP[G_c] ≥ 0       # Amount of new capacity for each canidate site 
    end)

    # Max and min generation constraints
    @constraint(CATS_Model, cMaxGenOld[g ∈ setdiff(G, G_c), t ∈ T], 
        GEN[g,t] ≤ variability[g,t]*gens[g,:pmax])
    @constraint(CATS_Model, cMaxGenNew[g ∈ G_c, t ∈ T], 
        GEN[g,t] ≤ variability[g,t]*(gens[g,:pmax]+CAP[g]))

    # Load shedding constraint
    @constraint(CATS_Model, cShed[i ∈ N, t ∈ T], 
        SHED[i,t] ≤ loads[i,t])
    
    #########################
    #### BATTERY STORAGE ####
    #########################
    # Charging must be less than max power capacity
    @constraint(CATS_Model, cMaxChargeOld[g ∈ setdiff(G_s, G_sc), t ∈ T], 
        CHARGE[g,t] ≤ gens[g,:pmax])
    @constraint(CATS_Model, cMaxChargeNew[g ∈ G_sc, t ∈ T], 
        CHARGE[g,t] ≤ gens[g,:pmax]+CAP[g])
    
    # Energy capacity must be less ×4 the power capacity
    @constraint(CATS_Model, cMaxStateOfChargeOld[g ∈ setdiff(G_s, G_sc), t ∈ T], 
        SOC[g,t] ≤ storage_hrs*gens[g,:pmax])
    @constraint(CATS_Model, cMaxStateOfChargeNew[g ∈ G_sc, t ∈ T], 
        SOC[g,t] ≤ storage_hrs*(gens[g,:pmax]+CAP[g]))
    
    # SOC in the next time is a function of SOC ine the pervious time
    # with circular wrapping for the first and last t ∈ P_i
    @constraint(CATS_Model, cStateOfCharge[g ∈ G_s, t ∈ T],
        SOC[g,t] == SOC[g,prev(t)] + (CHARGE[g,t]*η_charge - GEN[g,t]/η_discharge))
    
    ########################
    ######## DC OPF ########
    ########################
    # Create slack bus with theta=0
    fix.(THETA[slack_bus, T], 0)
    
    # Max line flow must be less than the nominal (RATE A) of the line
    @constraint(CATS_Model, cLineLimits[(i,j) ∈ L, t ∈ T], 
        FLOW[(i,j),t] ≤ line(i,j,:rate_a)) 
    
    # Angle limited to less than 60 degrees 
    @constraint(CATS_Model, cAngleLimitsMax[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≤  θlim)
    @constraint(CATS_Model, cAngleLimitsMin[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≥ -θlim)
                    
    # Flow is governed by angle difference between buses
    @constraint(CATS_Model, cLineFlows[(i,j) ∈ L, t ∈ T],
        FLOW[(i,j),t] == line(i,j,:sus)*(THETA[i,t] - THETA[j,t]));

    # Power must be balanced at all nodes
    # Σ(generation) + shedding - Σ(demand) - Σ charging = Σ(flows) ∀ n ∈ N
    @constraint(CATS_Model, cBalance[i ∈ N, t ∈ T], 
        sum(GEN[g,t] for g ∈ sel(G, i))
        + SHED[i,t] - loads[i,t] 
        - sum(CHARGE[g,t] for g ∈ sel(G_s, i))
         == sum(FLOW[(i,j),t] for j ∈ J(i)))

    #########################
    ####### OBJECTIVE #######
    #########################

    # Objective function (Note: Using just the linear part of quadratic cost)
    @objective(CATS_Model, Min,
            sum(gens[g,:c0] + gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T) 
            + sum(shed_cost * SHED[i,t] for i ∈ N, t ∈ T) #TODO expressions
    ) 
    # The weighted operational costs of running each generator
    @expression(CATS_Model, eVariableCosts,
        sum(W[t] * gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T))
    
    # The weighted operational costs of shedding load/non-served energy (NSE)
    @expression(CATS_Model, eNSECosts,
        sum(W[t] * shed_cost * SHED[i,t] for i ∈ N, t ∈ T))

    # Fixed costs of all solar investments
	@expression(CATS_Model, eFixedCostsSolar,
        sum(solar_cost * CAP[g] for g ∈ setdiff(G_c, G_sc)))

    # Fixed costs of all battery/ESS investments
    @expression(CATS_Model, eFixedCostsStorage,
		sum(battery_cost * CAP[g] for g ∈ G_sc))	

    # Minimize the sum of investment and O&M costs 
	@expression(CATS_Model, eTotalCosts,
		eFixedCostsSolar + eFixedCostsStorage + eVariableCosts + eNSECosts)
    
	@objective(CATS_Model, Min, eTotalCosts)

    optimize!(CATS_Model)

    return CATS_Model
end

basic_capex (generic function with 1 method)

In [37]:
# 3b. Ramping

# TODOs
# - kmeans clustering
# - Fix weighting
# - Ramping (if time)
# - Save more model outputs

## EXTRA
# Quardatic costs



## Questions for Qi ##
# a) costs? How do you handel the fact that costs are quadratic? Also why do they include c0?
# b) 1e-8 * 100 * (2700000 / 20 + 12500) # What are the units of this? and why do batteries cost less than solar??

In [39]:
run_model(load, basic_capex, save, "san_diego_system", "test_senario")

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1116864 rows, 480622 columns and 2386374 nonzeros
Model fingerprint: 0xa1f38981
Coefficient statistics:
  Matrix range     [1e-03, 1e+07]
  Objective range  [7e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-03, 3e+03]
Presolve removed 841147 rows and 247760 columns
Presolve time: 1.07s
Presolved: 275717 rows, 262262 columns, 988557 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 3.19s
Elapsed ordering time = 5s
Ordering time: 6.13s

Barrier statistics:
 Dense cols : 336
 Free vars  : 5458
 AA' NZ     : 1.697e+06
 Factor NZ  : 1.976e+07 (roughly 400 MB of

"/Users/adamsedlak/Documents/Julia/MAE243/final-project/models/san_diego_system/outputs/test_senario/prices.csv"

In [4]:
(buses, lines, gens, loads, variability, P) = load("san_diego_system");

In [41]:
161.71/60

2.6951666666666667